In [9]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

In [10]:
#Vorlage: https://www.tensorflow.org/tutorials/structured_data/feature_columns
dataframe = pd.read_csv('adult_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'adult_train.csv'

In [ ]:
dataframe.head()

In [ ]:
dataframe['target'] = np.where(dataframe['class']=='>50K', 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['fnlwgt', 'class'])

In [ ]:
#TODO die Fragezeichenwerte aus den Daten rausfiltern oder damit trainieren lassen?

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
dataframe.head()

In [ ]:
feature_columns = []

# numeric cols
for header in ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']:
  feature_columns.append(feature_column.numeric_column(header))

# indicator_columns
indicator_column_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship',
                          'race', 'sex', 'native-country']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataframe, test_size=0.2)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

print(len(train))

In [ ]:
tiny_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

tiny_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


tiny_model.fit(train_ds, validation_data=val_ds, epochs=100)

In [ ]:
small_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])

small_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


small_model.fit(train_ds, validation_data=val_ds, epochs=100)

In [ ]:
medium_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

medium_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


medium_model.fit(train_ds, validation_data=val_ds, epochs=100)

In [ ]:
large_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1)
])

large_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


large_model.fit(train_ds, validation_data=val_ds, epochs=100)

In [ ]:
history = {}
optimizer = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
#optimizer = ['Adam']
for x in optimizer:
    print(x)
    model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
    ])
    model.compile(optimizer=x,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history[x] = model.fit(train_ds, validation_data=val_ds, epochs=100)
    
    print()
    

In [ ]:
#TODO Vergleich optimierte Attribute gegen unveränderte
#TODO experimentieren wie das NN verbessert werden kann: verschiedene Netzwerkgrößen, Epochgröße, Optimizer, activation Funktion
#TODO save model
#TODO nochmal anschauen wie die Ausgaben aufbereitet werden können: print(history['Adam'].history)

In [ ]:
#load testdata 
dataframe_test = pd.read_csv('adult_test.csv')

In [ ]:
#prepare testdata
dataframe_test['target'] = np.where(dataframe_test['class']=='>50K', 0, 1)
dataframe_test = dataframe_test.drop(columns=['fnlwgt', 'class'])
test_ds = df_to_dataset(dataframe_test, shuffle=False, batch_size=batch_size)

In [ ]:
#Für Abschluss Validierung an ungesehenen Datensatz

#loss, accuracy = model.evaluate(test_ds)
#print("Accuracy", accuracy)